In [ ]:
# import packages
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms, utils
from torch.utils.data import TensorDataset
from tqdm import tqdm
from torch_geometric.nn import TransformerConv, global_max_pool, GATv2Conv, PointNetConv, ClusterGCNConv, PointTransformerConv, global_mean_pool
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
import time
import torch
from torch_geometric.data import Data, InMemoryDataset, download_url
import torch_geometric.transforms as T
import networkx as nx
from torch_cluster import knn_graph
from torch_geometric.utils import from_networkx

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


**Helper and Metric**


In [9]:
# Dataset class
import torch
from torch_geometric.data import Data, InMemoryDataset, download_url
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader

class Quark_Gluon_Dataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(Quark_Gluon_Dataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        pass

    def create_graph(self, image):
        G = nx.Graph()
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                if image[i][j].any() > 0:
                    #add node (i,j) to graph with attribute 'x' = image[i][j] and pos = (i,j)
                    G.add_node((i,j), x=image[i][j], pos=(i,j,0))
                    if i > 0 and image[i-1][j].any() > 0:
                        G.add_edge((i,j), (i-1,j))
                    if j > 0 and image[i][j-1].any() > 0:
                        G.add_edge((i,j), (i,j-1))
                    if i < image.shape[0]-1 and image[i+1][j].any() > 0:
                        G.add_edge((i,j), (i+1,j))
                    if j < image.shape[1]-1 and image[i][j+1].any() > 0:
                        G.add_edge((i,j), (i,j+1))
                    if i > 0 and j > 0 and image[i-1][j-1].any() > 0:
                        G.add_edge((i,j), (i-1,j-1))
                    if i < image.shape[0]-1 and j < image.shape[1]-1 and image[i+1][j+1].any() > 0:
                        G.add_edge((i,j), (i+1,j+1))
                    if i > 0 and j < image.shape[1]-1 and image[i-1][j+1].any() > 0:
                        G.add_edge((i,j), (i-1,j+1))
                    if i < image.shape[0]-1 and j > 0 and image[i+1][j-1].any() > 0:
                        G.add_edge((i,j), (i+1,j-1))

        return G

    def process(self):
        f = h5py.File('/hdfs1/Data/Shrutimoy/quark-gluon_data-set_n139306.hdf5', 'r')
        X_jets = np.asarray(f['X_jets'])
        m0 = np.asarray(f['m0'])
        pt = np.asarray(f['pt'])
        y = np.asarray(f['y'])
        data_list = []
        for i in tqdm(range(len(X_jets))):
            G = self.create_graph(X_jets[i])
            data = from_networkx(G)
            data.y = torch.tensor(y[i], dtype=torch.long)
            data.m = torch.tensor(m0[i], dtype=torch.float)
            data.p = torch.tensor(pt[i], dtype=torch.float)
            data_list.append(data)

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])


In [4]:
def train_test_split(X, y, test_size):
    dataset_size = len(X)
    train_data_X = X[:int(dataset_size*(1-test_size))]
    test_data_X = X[int(dataset_size*(1-test_size)):]
    train_data_y = y[:int(dataset_size*(1-test_size))]
    test_data_y = y[int(dataset_size*(1-test_size)):]
    return train_data_X, test_data_X, train_data_y, test_data_y


In [5]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


**Model**


In [10]:
# Define the model
class GraphClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout):
        super(GraphClassifier, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.convs_trans = nn.ModuleList()
        self.convs_point = nn.ModuleList()
        self.convs_attn = nn.ModuleList()
        self.fc1 = nn.ModuleList()
        self.batch_norm = nn.ModuleList()
        for i in range(num_layers):
            self.convs_trans.append(GATv2Conv(hidden_dim*(3**i), hidden_dim*(3**i), add_self_loops=True))
            self.convs_point.append(PointTransformerConv(hidden_dim*(3**i), hidden_dim*(3**i),  add_self_loops=True))
            self.convs_attn.append(ClusterGCNConv(hidden_dim*(3**i), hidden_dim*(3**i), add_self_loops=True))
            self.batch_norm.append(nn.BatchNorm1d(hidden_dim*(3**(i+1))))
            self.fc1.append(nn.Linear(hidden_dim*(3**(i+1)), hidden_dim*(3**(i+1))))
        #self.fc1 = nn.Linear(hidden_dim*3, hidden_dim)
        self.fc0 = nn.Linear(input_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.batch_norm1 = nn.BatchNorm1d(hidden_dim*(3**num_layers)+2)
        self.fc2 = nn.Linear(hidden_dim*(3**num_layers)+2, hidden_dim*(3**num_layers)+2)
        self.fc3 = nn.Linear(hidden_dim*(3**num_layers)+2, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x, edge_index, pos, batch, m0, pt):
        pos = pos.float()
        x = self.fc0(x)
        for i in range(self.num_layers):
            x1 = self.convs_point[i](x, pos, edge_index)
            x1 = F.elu(x1)
            x2 = self.convs_attn[i](x, edge_index)
            x2 = F.elu(x2)
            x3 = self.convs_trans[i](x, edge_index)
            x3 = F.elu(x3)
            x = torch.cat([x1, x2, x3], dim=1)
            x = self.batch_norm[i](x)
            x = self.fc1[i](x)
        x = global_mean_pool(x, batch)
        m0 = m0.reshape(-1, 1)
        pt = pt.reshape(-1, 1)
        x = torch.cat([x, m0, pt], dim=1)
        x = self.fc2(x)
        x = F.elu(x)
        x = self.batch_norm1(x)
        x = self.fc3(x)
        return self.softmax(x)


**Main**


In [11]:
dataset = Quark_Gluon_Dataset(root='/hdfs1/Data/Shubhajit/Quark_Gluon_Data_1/')


In [12]:
# split dataset into train, validation and test
train_dataset = dataset[:int(0.8*len(dataset))]
val_dataset = dataset[int(0.8*len(dataset)):int(0.9*len(dataset))]
test_dataset = dataset[int(0.9*len(dataset)):]

# load the dataset
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [13]:
#initialize the model
model = GraphClassifier(3, 16,2,3, 0).to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.50)

In [14]:
model.parameters

<bound method Module.parameters of GraphClassifier(
  (convs_trans): ModuleList(
    (0): GATv2Conv(16, 16, heads=1)
    (1): GATv2Conv(48, 48, heads=1)
    (2): GATv2Conv(144, 144, heads=1)
  )
  (convs_point): ModuleList(
    (0): PointTransformerConv(16, 16)
    (1): PointTransformerConv(48, 48)
    (2): PointTransformerConv(144, 144)
  )
  (convs_attn): ModuleList(
    (0): ClusterGCNConv(16, 16, diag_lambda=0.0)
    (1): ClusterGCNConv(48, 48, diag_lambda=0.0)
    (2): ClusterGCNConv(144, 144, diag_lambda=0.0)
  )
  (fc1): ModuleList(
    (0): Linear(in_features=48, out_features=48, bias=True)
    (1): Linear(in_features=144, out_features=144, bias=True)
    (2): Linear(in_features=432, out_features=432, bias=True)
  )
  (batch_norm): ModuleList(
    (0): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm1d(432, eps=1e-05, momentum=0.1, affine=T

In [15]:
# training loop
def train(data):
    optimizer.zero_grad()
    data = data.to(device)
    output = model(data.x, data.edge_index, data.pos, data.batch, data.m, data.p)
    loss_train = F.cross_entropy(output, data.y)
    loss_train.backward()
    optimizer.step()
    return output, loss_train


def test(data):
    data = data.to(device)
    output = model(data.x, data.edge_index, data.pos, data.batch, data.m, data.p)
    loss_test = F.cross_entropy(output, data.y)
    return output, loss_test


In [16]:


for epoch in range(100):
    start = time.time()
    # Train for one epoch
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    step = 0
    for data in tqdm(train_loader):
        output, loss = train(data)
        train_loss.update(loss.item(), output.size(0))
        train_acc.update(accuracy(output, data.y), output.size(0))

    # Evaluate on validation set
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    for data in tqdm(val_loader):
        output, loss = test(data)
        val_loss.update(loss.item(), output.size(0))
        val_acc.update(accuracy(output, data.y), output.size(0))

    # Evaluate on test set
    test_loss = AverageMeter()
    test_acc = AverageMeter()
    for data in tqdm(test_loader):
        output, loss = test(data)
        test_loss.update(loss.item(), output.size(0))
        test_acc.update(accuracy(output, data.y), output.size(0))

    print("epoch:" + '%03d ' % (epoch + 1) + "train_loss=" + "{:.5f} ".format(train_loss.avg) + "train_acc=" + "{:.5f} ".format(train_acc.avg) +
          "val_acc=" + "{:.5f} ".format(val_acc.avg) + "test_acc=" + "{:.5f} ".format(test_acc.avg) + "time=" + "{:.5f} ".format(time.time() - start))

    scheduler.step()

    # log the results tom tensorboard
    writer.add_scalar('train_loss', train_loss.avg, epoch)
    writer.add_scalar('train_acc', train_acc.avg, epoch)
    writer.add_scalar('val_loss', val_loss.avg, epoch)
    writer.add_scalar('val_acc', val_acc.avg, epoch)
    writer.add_scalar('test_loss', test_loss.avg, epoch)
    writer.add_scalar('test_acc', test_acc.avg, epoch)


100%|██████████| 871/871 [00:17<00:00, 48.43it/s]


epoch:001 train_loss=0.58725 train_acc=0.70798 val_acc=0.57333 test_acc=0.57419 time=354.22800 


100%|██████████| 871/871 [00:12<00:00, 67.93it/s]


epoch:002 train_loss=0.58253 train_acc=0.71266 val_acc=0.68430 test_acc=0.67913 time=321.55234 


100%|██████████| 871/871 [00:13<00:00, 65.65it/s]


...

100%|██████████| 871/871 [00:19<00:00, 45.61it/s]


epoch:050 train_loss=0.56348 train_acc=0.73471 val_acc=0.72026 test_acc=0.71481 time=339.90290 


100%|██████████| 871/871 [00:14<00:00, 59.01it/s]


epoch:051 train_loss=0.56313 train_acc=0.73545 val_acc=0.73713 test_acc=0.73225 time=334.01418 


  7%|▋         | 489/6966 [00:20<04:41, 23.01it/s]